# Visualize the data 

In [11]:
#coding=utf-8
import json
import geopandas
import matplotlib.pyplot as plt
import requests
import pandas as pd
from shapely.geometry import LineString
import time
import os
import numpy as np
import matplotlib as mpl
import seaborn as sns#添加Seaborn模块

In [2]:
 def plot_show(data) :
    data.plot(column='speed', cmap='RdYlGn', legend=True)
    plt.show()

In [6]:
import os
filePath = '/Users/jackyanghc/Desktop/Jupyter/'
filename = []
for i,j,k in os.walk(filePath):
    for m in j:
        if m[0] == 'S':
            filename.append(m)

ShangHai = geopandas.read_file(filePath+filename[0],encoding='utf-8')
for i in range(1,len(filename)):
    tmp = geopandas.read_file(filePath+filename[i],encoding='utf-8')
    ShangHai = pd.concat([ShangHai,tmp])

print(ShangHai)

         name            Timestamp  \
0       申嘉湖高速  2019_11_20 19:02:00   
1       申嘉湖高速  2019_11_20 19:02:00   
2      上海绕城高速  2019_11_20 19:02:00   
3      上海绕城高速  2019_11_20 19:02:00   
4       ３２４省道  2019_11_20 19:02:00   
...       ...                  ...   
92128     华申路  2019_11_21 12:00:00   
92129     华申路  2019_11_21 12:00:00   
92130    五洲大道  2019_11_21 12:00:00   
92131     申江路  2019_11_21 12:00:00   
92132     东靖路  2019_11_21 12:00:00   

                                                   Lcode  speed Congestion  \
0      4054,4056,4061,4063,4604,4066,4069,4070,4072,4...     95          1   
1      -4084,-4083,-4079,-4078,-4076,-4072,-4070,-406...     95          1   
2      2011,2010,2009,2008,2007,2006,2005,2392,2001,2000     80          1   
3      -1994,-1995,-1996,-1998,-2000,-2001,-2392,-200...     80          1   
4      4096,4098,4106,4107,4110,4116,4127,4128,4129,4...     45          1   
...                                                  ...    ...        ... 

In [7]:
ShangHai = ShangHai.drop_duplicates(
    keep='first',
    inplace=False)
ShangHai = ShangHai.reset_index(drop=True)

In [8]:
information = ShangHai[['name','Timestamp','Lcode','speed','Congestion']]
information

,name,Timestamp,Lcode,speed,Congestion
0,申嘉湖高速,2019_11_20 19:02:00,"4054,4056,4061,4063,4604,4066,4069,4070,4072,4...",95,1
1,申嘉湖高速,2019_11_20 19:02:00,"-4084,-4083,-4079,-4078,-4076,-4072,-4070,-406...",95,1
2,上海绕城高速,2019_11_20 19:02:00,"2011,2010,2009,2008,2007,2006,2005,2392,2001,2000",80,1
3,上海绕城高速,2019_11_20 19:02:00,"-1994,-1995,-1996,-1998,-2000,-2001,-2392,-200...",80,1
4,３２４省道,2019_11_20 19:02:00,"4096,4098,4106,4107,4110,4116,4127,4128,4129,4...",45,1
...,...,...,...,...,...
996408,富特东三路,2019_11_21 12:00:00,"-5100,-5099",45,1
996409,日樱南路,2019_11_21 12:00:00,"2425,2427,2430",35,1
996410,日樱南路,2019_11_21 12:00:00,"-2427,-2425",40,1
996411,庭安路,2019_11_21 12:00:00,"-6135,-6134",35,1


In [13]:
# 将信息进行冗余查询
for i in range(0,10):
    tmp = information.loc[i]
    codes = tmp['Lcode'].split(',')
    number = len(codes)
    if number == 1:
        continue
    else:
        information.loc[i]['Lcode'] = codes[0]
        print(i)
        for j in range(1,number):
            tmp.loc['Lcode'] = codes[j]
            information = information.append(tmp)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


0
1
2
3
4
5
6
7
8
9


# 针对一条道路进行全天的速度和堵塞进行可视化

In [12]:
from pyecharts import options as opts
from pyecharts.charts import Bar


In [9]:
def select_on_Lcode(lcode):
    Road_info_time = []
    Road_info_speed = []
    Road_info_congest = []
    Road_info_lcode = [lcode]
    for i in range(0,len(information)):
        if lcode in information.loc[i,'Lcode'].split(","):
            tmp = information.loc[i].values
            Road_info_time.append(tmp[1])
            Road_info_speed.append(tmp[3])
            Road_info_congest.append(tmp[4])
    return [Road_info_lcode,Road_info_time,Road_info_speed,Road_info_congest]

In [10]:
info = select_on_Lcode('-608')

In [13]:
def bar_base(name_y,x,y,title == None,subtitle == None ) -> Bar:
    c = (
        Bar()
        .add_xaxis(x)
        .add_yaxis(name_y[0],y[0])
        .set_global_opts(title_opts=opts.TitleOpts(title= title, subtitle=subtitle))
    )
    return c
bar_base(info[0],info[1],info[2]).render_notebook()

SyntaxError: invalid syntax (<ipython-input-13-0000592d27cc>, line 1)